# Imorting Libraries

In [2]:
#Before running this project, ensure you have the necessary libraries installed. Below are the required imports:

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
import matplotlib.pyplot as plt
from PIL import Image  # Import PIL to resize images
import shutil
import seaborn as sns

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset,DataLoader, random_split
import random
from sklearn.model_selection import train_test_split
from torchmetrics.functional import structural_similarity_index_measure as ssim

import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import classification_report, confusion_matrix

import seaborn as sns
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ColorJitter
if torch.cuda.is_available():
    device = torch.device("cuda")  # Set device to GPU
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("CUDA is not available. Using CPU.")

Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
import lpips
import torch
from PIL import Image
import torchvision.transforms as transforms

# Initialize the LPIPS model (default uses AlexNet backbone)
loss_fn = lpips.LPIPS(net='alex')  # Choices: 'alex', 'vgg', 'squeeze'

# Load real and adversarial images
def load_image(image):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize to match input size for LPIPS
        transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
    ])
    return transform(image)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\david\anaconda3\envs\MOSMED\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\david\anaconda3\envs\MOSMED\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\david\anaconda3\envs\MOSMED\Lib\site-packages\lpips\weights\v0.1\alex.pth


C:\Users\david\anaconda3\envs\MOSMED\Lib\site-packages\lpips\lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_loca

# Data Preprocessing

In [4]:
transform = transforms.Compose([
        transforms.Normalize([0.5], [0.5])
    ])

# Load CNN

In [5]:
import torch
import torch.nn as nn
from torchvision import models

# Define the model class (ResNet18 for single-channel input)
def get_resnet18_model(num_classes):
    model = models.resnet18(pretrained=False)  # pretrained=False as we load custom weights
    model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Function to load the model
def load_model(model, path):
    model.load_state_dict(torch.load(path))
    model.eval()  # Set the model to evaluation mode
    print(f'Model loaded from {path}')

# Specify the number of classes
num_classes = 4  # COVID-19, Normal, Lung Opacity, Viral Pneumonia

# Initialize the model and load the trained weights
model = get_resnet18_model(num_classes)

C:\Users\david\anaconda3\envs\MOSMED\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
state_dict = torch.load('CNN.pth')

C:\Users\david\AppData\Local\Temp\ipykernel_29028\750257864.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('CNN.pth')


In [7]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Load MLP

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SVDModel(nn.Module):
    def __init__(self,input_size,output_size):
        super(SVDModel, self).__init__()
        #layers
        # self.fc1 = nn.Linear(input_size,512)
        # self.fc2 = nn.Linear(512,256)
        # self.fc3 = nn.Linear(256,output_size)
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),  
            nn.ReLU(),  
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
            nn.Linear(64,output_size)
        )
    def forward(self, x):
        # Define the forward pass
        # x = F.relu(self.fc1(x))  # Apply ReLU after first layer
        # x = F.relu(self.fc2(x))  # Apply ReLU after second layer
        # x = self.fc3(x)          # Final layer, no activation (linear output)
        x = self.model(x)
        return x
        

In [10]:
sub_model = torch.load('MLP.pth')

C:\Users\david\AppData\Local\Temp\ipykernel_29028\4073176093.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sub_model = torch.load('MLP.pth')


In [11]:
sub_model.eval()

SVDModel(
  (model): Sequential(
    (0): Linear(in_features=174, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU()
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): ReLU()
    (10): Linear(in_features=256, out_features=256, bias=True)
    (11): ReLU()
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): ReLU()
    (14): Linear(in_features=128, out_features=128, bias=True)
    (15): ReLU()
    (16): Linear(in_features=128, out_features=64, bias=True)
    (17): ReLU()
    (18): Linear(in_features=64, out_features=64, bias=True)
    (19): ReLU()
    (20): Linear(in_features=64, out_features=174, bias=True)
  )
)

# Load Data(images)

In [12]:
U_adv_list = np.load('SVD_ADV_Images/U_adv_list.npy')
S_adv_list = np.load('SVD_ADV_Images/S_adv_list.npy')
V_adv_list = np.load('SVD_ADV_Images/V_adv_list.npy')

In [13]:
U_img_list = np.load('SVD_Real_Images/U_img_list.npy')
S_img_list = np.load('SVD_Real_Images/S_img_list.npy')
V_img_list = np.load('SVD_Real_Images/V_img_list.npy')

# Attack

In [14]:
def reconstruction(U,S,Vt):
    Sigma = np.zeros((224,224))# assuming image is 224x224
    np.fill_diagonal(Sigma, S)
    img = np.dot(U, np.dot(Sigma, Vt))
    # img = np.clip(img, 0, 1)
    # plt.imshow(img,cmap='gray')
    img = torch.tensor(img)
    img = img.unsqueeze(0)
    img = img.unsqueeze(0).float()
    return img

In [15]:
def S_attack(S,model):
    S = torch.tensor(S)
    S_174 = S[50:]
    out = model(S_174)
    S_new = np.zeros(224)
    S_new[:50] = S[:50]
    S_new[50:] = out.detach().numpy()
    return S_new

In [17]:
# Attack Block

adv_imgs=[]
imgs=[]
not_adv = []
lpips_scores=[]
ssim_scores=[]
ASR =[]
actual_pred = []
adv_pred = []
for i in range(0,10):
    a=20
    # S_atck
    S0 = S_img_list[i]
    
    S_atk = S_attack(S0,sub_model)
    # Reconstruction
    real_image = reconstruction(U_img_list[i],S_img_list[i],V_img_list[i])
    #3,224,224
    real_image_224 = real_image
    # Looping
    pred = torch.max(model(real_image.to(device)).data,1)[1].item()
    loop=1
    pred1=pred
    while pred1==pred:
        if loop>=3000:
            print('Too many interations')
            break
        print('loop: ',loop)
        S_atk[:] = S_atk[:] + (S_atk[:]/a)
        
        atk_image = reconstruction(U_img_list[i],S_atk,V_img_list[i])
        # try_image = transform(try_image)        
        atk_1 = model(atk_image.to(device))
        print(atk_1)
        pred1 = torch.max(atk_1.data,1)[1]
        loop+=1
        if a<800:
            a=a*1.1
    if pred1!=pred:
        adv_imgs.append(atk_image)
        imgs.append(real_image)
        ssim_real_img = (real_image - real_image.min())/(real_image.max()-real_image.min())
        ssim_adv_img = (atk_image - atk_image.min())/(atk_image.max()-atk_image.min())
        ssim_score = ssim(ssim_real_img,ssim_adv_img,data_range=1.0)
        image = load_image(ssim_real_img)
        adv_image = load_image(ssim_adv_img)
        lpips_distance = loss_fn(image, adv_image)
        if ssim_score>=0.86:
            gray_1 = atk_image.squeeze(0).squeeze(0)
            gray_2 = real_image.squeeze(0).squeeze(0)
            plt.imsave("Sample Attacked Images/Class 0 - COVID/"+str(i)+".Real_Image.png", gray_2.numpy(), cmap="gray")
            plt.imsave("Sample Attacked Images/Class 0 - COVID/"+str(i)+".Adv_Image(SSIM = "+str(ssim_score)+", LPIPS = "+str(lpips_distance.item())+").png", gray_1.numpy(), cmap="gray")
        actual_pred.append(pred)
        adv_pred.append(pred1)
        ssim_scores.append(ssim_score)
        ASR.append(1)
        
        lpips_scores.append(lpips_distance)
    else:
        ASR.append(0)
        not_adv.append(atk_image)
    
    print(i+1,' images done')
    print('#########################################################################################################################################')

loop:  1
tensor([[10.6106, -0.2973, -7.3021, -0.1622]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  2
tensor([[10.4607, -0.3768, -7.1890, -0.1174]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  3
tensor([[10.2594, -0.4443, -7.0495, -0.0442]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  4
tensor([[10.0820, -0.5055, -6.8990,  0.0159]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  5
tensor([[ 9.9188, -0.5699, -6.7480,  0.0713]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  6
tensor([[ 9.8052, -0.6157, -6.6266,  0.1154]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  7
tensor([[ 9.6930, -0.6672, -6.4920,  0.1460]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  8
tensor([[ 9.6178, -0.7133, -6.3607,  0.1408]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  9
tensor([[ 9.5681, -0.7600, -6.2398,  0.1137]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  10
tensor([[ 9.5182, -0.7709, -6.1431, 

C:\Users\david\anaconda3\envs\MOSMED\Lib\site-packages\torchmetrics\utilities\prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(


1  images done
#########################################################################################################################################
loop:  1
tensor([[ 6.4112,  0.5326, -1.3706, -3.7358]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  2
tensor([[ 6.4539,  0.5712, -1.4085, -3.7362]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  3
tensor([[ 6.5127,  0.5897, -1.4103, -3.7634]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  4
tensor([[ 6.5394,  0.6028, -1.3610, -3.8429]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  5
tensor([[ 6.5889,  0.5946, -1.3172, -3.9265]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  6
tensor([[ 6.6058,  0.5857, -1.2608, -3.9893]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  7
tensor([[ 6.6330,  0.5773, -1.2185, -4.0473]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
loop:  8
tensor([[ 6.6398,  0.5863, -1.1791, -4.0956]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

# Attack Evaluation


In [18]:
#SSIM
sum(ssim_scores)/len(ssim_scores) # Average SSIM

tensor(0.9388)

In [19]:
ssim_scores

[tensor(0.9252),
 tensor(0.9432),
 tensor(0.9475),
 tensor(0.9014),
 tensor(0.9395),
 tensor(0.9592),
 tensor(0.9554)]

In [20]:
#ASR
sum(ASR)/len(ASR)

0.7

In [21]:
#LPIPS
sum(lpips_scores)/len(lpips_scores) #Average LPIPS

tensor([[[[0.1024]]]], grad_fn=<DivBackward0>)

In [22]:
lpips_scores

[tensor([[[[0.1431]]]], grad_fn=<AddBackward0>),
 tensor([[[[0.1351]]]], grad_fn=<AddBackward0>),
 tensor([[[[0.0705]]]], grad_fn=<AddBackward0>),
 tensor([[[[0.1554]]]], grad_fn=<AddBackward0>),
 tensor([[[[0.0642]]]], grad_fn=<AddBackward0>),
 tensor([[[[0.1059]]]], grad_fn=<AddBackward0>),
 tensor([[[[0.0429]]]], grad_fn=<AddBackward0>)]